In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import xarray as xr
import hsmm_mvpy as hmp
from pathlib import Path
import matplotlib.pyplot as plt
import itertools
import multiprocessing
import os
import random
import mne

In [3]:
cpus = multiprocessing.cpu_count()
print(cpus)
cpus = 1  # if not enough RAM to load all participants data, better reduce the number of processes

path = "estimation_files/"
starting_points = 1
version = "1Hz"

force_id = {"low": 1, "high": 2}
stim_id = {
    "accuracy/left/1": 211,
    "accuracy/left/2": 212,
    "accuracy/left/3": 213,
    "accuracy/right/1": 221,
    "accuracy/right/2": 222,
    "accuracy/right/3": 223,
    "speed/left/1": 111,
    "speed/left/2": 112,
    "speed/left/3": 113,  # conditions in the experiment
    "speed/right/1": 121,
    "speed/right/2": 122,
    "speed/right/3": 123,
}  # used for segmentation
resp_id = {"r_left": 100, "r_right": 200}  # Response events
event_id = {
    "low/accuracy/left/1": 1211,
    "low/accuracy/left/2": 1212,
    "low/accuracy/left/3": 1213,
    "low/accuracy/right/1": 1221,
    "low/accuracy/right/2": 1222,
    "low/accuracy/right/3": 1223,
    "low/speed/left/1": 1111,
    "low/speed/left/2": 1112,
    "low/speed/left/3": 1113,  # conditions in the experiment
    "low/speed/right/1": 1121,
    "low/speed/right/2": 1122,
    "low/speed/right/3": 1123,
    "high/accuracy/left/1": 2211,
    "high/accuracy/left/2": 2212,
    "ahigh/ccuracy/left/3": 2213,
    "high/accuracy/right/1": 2221,
    "high/accuracy/right/2": 2222,
    "high/accuracy/right/3": 2223,
    "high/speed/left/1": 2111,
    "high/speed/left/2": 2112,
    "high/speed/left/3": 2113,  # conditions in the experiment
    "high/speed/right/1": 2121,
    "high/speed/right/2": 2122,
    "high/speed/right/3": 2123,
}

sfreq = 100  # For resampling

eeg_path = Path("../data/sat2/preprocessed_500hz/")
subj_names = [
    x.split("_")[1].split(".")[0] for x in os.listdir(eeg_path) if 'S14' not in x
]  # in x and 'S14' not in x] #pbm with
# subj_names = subj_names[0:5]
print(subj_names)
subj_files = [str(eeg_path) + "/preprocessed_" + x + "_raw.fif" for x in subj_names]
# Filter out first two participants for testing

24
['S10', 'S11', 'S12', 'S13', 'S15', 'S16', 'S17', 'S18', 'S1', 'S20', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9']


In [4]:
save_file = "epoch_data_FEEG_500Hz.nc"
events_corrected = []
epoch_data = []
for participant in subj_files:
    raw = mne.io.read_raw_fif(participant, preload=False, verbose=False)
    events = mne.find_events(raw, verbose=False)
    events[:, 2] = events[:, 2] - events[:, 1]  # correct events values
    i = 0
    while i < len(events):
        if events[i, 2] in force_id.values():  # if events is condition trigger
            events[i + 1, 2] = int(
                str(events[i, 2]) + str(events[i + 1, 2])
            )  # places condition trigger on stimulus trigger (no photodiode used yet)
            events[i, 2] = 0
            i += 2
        else:
            i += 1
    events_values = np.concatenate(
        [
            np.array([x for x in event_id.values()]),
            np.array([x for x in resp_id.values()]),
        ]
    )
    events_wresp = np.array([list(x) for x in events if x[2] in events_values])
    events_corrected.append(events_wresp)
    # print("Starting mp now")
    epoch_data_participant = hmp.utils.read_mne_data(
        pfiles=participant,
        event_id=event_id,
        resp_id=resp_id,
        epoched=False,
        sfreq=sfreq,
        subj_idx=None,
        metadata=None,
        events_provided=events_wresp,
        rt_col="rt",
        rts=None,
        verbose=True,
        tmin=-0.25,
        tmax=2.5,
        offset_after_resp=0,
        high_pass=1,
        low_pass=None,
        pick_channels="eeg",
        baseline=(None, 0),
        upper_limit_RT=2.5,
        lower_limit_RT=0.25,
    )
    print(epoch_data_participant)
    epoch_data.append(epoch_data_participant)


epoch_data = xr.concat(
    epoch_data,
    pd.Series(subj_names, name="participant"),
    fill_value={"event": "", "data": np.nan},
)  # not sure about the order of the particiapnt use with caution
# epoch_data = xr.open_dataset(
#     save_file
# )  # For an unknown reason I need to re-read it


# raw = mne.io.read_raw_fif(
#     "../data/sat2/preprocessed_500hz/preprocessed_S1_raw.fif",
#     preload=False,
#     verbose=False,
# )  # loading for sensor position
# raw.set_montage(mne.channels.make_standard_montage("biosemi64"))

Processing participant ../data/sat2/preprocessed_500hz/preprocessed_S10_raw.fif's continuous eeg
Opening raw data file ../data/sat2/preprocessed_500hz/preprocessed_S10_raw.fif...
    Range : 0 ... 4814847 =      0.000 ...  4701.999 secs
Ready.
Reading 0 ... 4814847  =      0.000 ...  4701.999 secs...
Downsampling to 100 Hz
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 33 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 33.03 Hz
- Upper transition bandwidth: 8.26 Hz (-6 dB cutoff frequency: 37.16 Hz)
- Filter length: 3381 samples (3.302 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.2s


Creating epochs based on following event ID :[ 100  200 1111 1112 1113 1121 1122 1123 1211 1212 1213 1221 1222 1223
 2111 2112 2113 2121 2122 2123 2211 2212 2213 2221 2222 2223]
Adding metadata with 2 columns
2448 matching events found
Setting baseline interval to [-0.244140625, 0.0] s
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 2448 events and 2817 original time points (prior to decimation) ...
169 bad epochs dropped
Applying reaction time trim to keep RTs between 0.25 and 2.5 seconds
2251 RTs kept of 2279 clean epochs
2251 trials were retained for participant ../data/sat2/preprocessed_500hz/preprocessed_S10_raw.fif
End sampling frequency is 100 Hz
<xarray.Dataset>
Dimensions:      (participant: 1, epochs: 2251, channels: 64, samples: 186)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 ... 2441 2442 2443 2444 2445 2446
  * channels     (channels) <U3 'Fp1' 'AF7' 'AF3' 'F1' ... 'PO8' 'PO4' 'O2'
  * samples      (samples) int64 0 1 2 3 4 5 6 7

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.3s


Creating epochs based on following event ID :[ 100  200 1111 1112 1113 1121 1122 1123 1211 1212 1213 1221 1222 1223
 2111 2112 2113 2121 2122 2123 2211 2212 2213 2221 2222 2223]
Adding metadata with 2 columns
2448 matching events found
Setting baseline interval to [-0.244140625, 0.0] s
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 2448 events and 2817 original time points (prior to decimation) ...
265 bad epochs dropped
Applying reaction time trim to keep RTs between 0.25 and 2.5 seconds
2104 RTs kept of 2183 clean epochs
2104 trials were retained for participant ../data/sat2/preprocessed_500hz/preprocessed_S11_raw.fif
End sampling frequency is 100 Hz
<xarray.Dataset>
Dimensions:      (participant: 1, epochs: 2104, channels: 64, samples: 249)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 ... 2441 2442 2443 2444 2445 2446
  * channels     (channels) <U3 'Fp1' 'AF7' 'AF3' 'F1' ... 'PO8' 'PO4' 'O2'
  * samples      (samples) int64 0 1 2 3 4 5 6 7

...

Creating epochs based on following event ID :[ 100  200 1111 1112 1113 1121 1122 1123 1211 1212 1213 1221 1222 1223
 2111 2112 2113 2121 2122 2123 2211 2212 2213 2221 2222 2223]
Adding metadata with 2 columns
2448 matching events found
Setting baseline interval to [-0.244140625, 0.0] s
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 2448 events and 2817 original time points (prior to decimation) ...
202 bad epochs dropped
Applying reaction time trim to keep RTs between 0.25 and 2.5 seconds
2044 RTs kept of 2246 clean epochs
2044 trials were retained for participant ../data/sat2/preprocessed_500hz/preprocessed_S7_raw.fif
End sampling frequency is 100 Hz
<xarray.Dataset>
Dimensions:      (participant: 1, epochs: 2044, channels: 64, samples: 250)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 ... 2441 2442 2443 2444 2445 2446
  * channels     (channels) <U3 'Fp1' 'AF7' 'AF3' 'F1' ... 'PO8' 'PO4' 'O2'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.2s


Creating epochs based on following event ID :[ 100  200 1111 1112 1113 1121 1122 1123 1211 1212 1213 1221 1222 1223
 2111 2112 2113 2121 2122 2123 2211 2212 2213 2221 2222 2223]
Adding metadata with 2 columns
2448 matching events found
Setting baseline interval to [-0.244140625, 0.0] s
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 2448 events and 2817 original time points (prior to decimation) ...
213 bad epochs dropped
Applying reaction time trim to keep RTs between 0.25 and 2.5 seconds
2077 RTs kept of 2235 clean epochs
2077 trials were retained for participant ../data/sat2/preprocessed_500hz/preprocessed_S8_raw.fif
End sampling frequency is 100 Hz
<xarray.Dataset>
Dimensions:      (participant: 1, epochs: 2077, channels: 64, samples: 250)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 ... 2428 2429 2437 2439 2443 2446
  * channels     (channels) <U3 'Fp1' 'AF7' 'AF3' 'F1' ... 'PO8' 'PO4' 'O2'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.3s


Creating epochs based on following event ID :[ 100  200 1111 1112 1113 1121 1122 1123 1211 1212 1213 1221 1222 1223
 2111 2112 2113 2121 2122 2123 2211 2212 2213 2221 2222 2223]
Adding metadata with 2 columns
2448 matching events found
Setting baseline interval to [-0.244140625, 0.0] s
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 2448 events and 2817 original time points (prior to decimation) ...
65 bad epochs dropped
Applying reaction time trim to keep RTs between 0.25 and 2.5 seconds
2348 RTs kept of 2383 clean epochs
2348 trials were retained for participant ../data/sat2/preprocessed_500hz/preprocessed_S9_raw.fif
End sampling frequency is 100 Hz
<xarray.Dataset>
Dimensions:      (participant: 1, epochs: 2348, channels: 64, samples: 205)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 ... 2441 2442 2443 2444 2445 2446
  * channels     (channels) <U3 'Fp1' 'AF7' 'AF3' 'F1' ... 'PO8' 'PO4' 'O2'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 .

In [6]:
epoch_data_sat1 = xr.load_dataset(Path("../data/sat1/data_100hz.nc"))
epoch_data_sat1

<xarray.Dataset>
Dimensions:      (epochs: 200, channels: 30, samples: 199, participant: 25)
Coordinates:
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * channels     (channels) object 'Fp1' 'Fp2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 192 193 194 195 196 197 198
    stim         (participant, epochs) float64 nan 1.0 1.0 1.0 ... 2.0 nan 2.0
    resp         (participant, epochs) object '' 'resp_left' ... '' 'resp_left'
    RT           (participant, epochs) float64 nan 683.0 ... nan 1.02e+03
    cue          (participant, epochs) object '' 'SP' 'AC' 'SP' ... 'SP' '' 'AC'
    movement     (participant, epochs) object '' 'stim_left' ... '' 'stim_right'
    trigger      (participant, epochs) object '' ... 'AC/stim_right/resp_left'
  * participant  (participant) object '0001' '0002' '0003' ... '0024' '0025'
Data variables:
    data         (participant, epochs, channels, samples) float64 nan ... nan
Attributes:
    sfreq:           100.0
    offset:          0
    lowpass:         35.0
    highpass:        1.0
    lower_limit_RT:  0.2
    upper_limit_RT:  2

In [5]:
epoch_data = epoch_data.sel(channels=epoch_data_sat1.channels)
epoch_data.to_netcdf(Path("../data/sat2/data.nc"))